In [27]:
#Declare the Libraries
import os
#import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
#import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
#from sklearn.cross_validation import train_test_split  
from sklearn.model_selection import train_test_split  
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pickle

In [28]:
#import data
input_data= pd.read_csv("C:\\Users\\BENSHE\\Desktop\\Training\\Data_Science\\Data_Mining_Data_Pipeline\\Naive Bayes\\deceptive-opinion.csv")

In [29]:
#Shape & describe
input_data.shape

(1600, 5)

In [ ]:
#Shape & describe
input_data.describe(include='all')

In [ ]:
#Info of Dataframe
input_data.info()

In [ ]:
input_data.head(5)

In [30]:
input_data['org_text']=input_data['text']

In [31]:
#Change to Lower Case
input_data['text']= input_data['text'].str.lower()

In [32]:
#Remove URL
input_data['text'] = input_data['text'].str.replace('http\S+|www.\S+', ' ', case=False)

In [33]:
#Remove Numbers
input_data['text']=input_data['text'].str.replace('\d+', ' ')

In [34]:
#Remove special characters
input_data['text']=input_data['text'].str.replace('\W', ' ')

In [35]:
#Remove double space
input_data['text']=input_data['text'].str.replace('\s{3,}', ' ')
input_data['text']=input_data['text'].str.replace('\s{2,}', ' ')

In [36]:
#Lammatization using textblob

from textblob import TextBlob, Word

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  
    #dictionary.get(keyname, value)
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [37]:
#Apply for Lammetization
input_data['text']=input_data.text.apply(lemmatize_with_postag)

In [38]:
#NLP Pipeline
import nltk

In [39]:
#download the NLTK stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BENSHE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
# Import stopwords with scikit-learn
from sklearn.feature_extraction import text
stop_words_sklearn = text.ENGLISH_STOP_WORDS

In [41]:
#Create List of stop words
stop_words=nltk.corpus.stopwords.words('english')

In [42]:
#Words to be retained and not skipped
retain_words=['i','my']

In [43]:
#remove important words if any from the stop words list
for word in retain_words:
    stop_words.remove(word)

In [44]:
#Add skip words or additional stop words
additional_stopwords = ['stop','rare','xyz']
for word in additional_stopwords:
    stop_words.extend(additional_stopwords)

In [45]:
#Remove the stop words from the text
for word in stop_words :
    input_data['text'] = input_data['text'].replace(to_replace=r'\b%s\b'%word, value="",regex=True)

In [46]:
#Remove leading and trailing spaces
input_data['text']=input_data['text'].map(lambda x: x.strip())

In [47]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BENSHE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
#Downlaod NLTK Punkt package
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BENSHE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\BENSHE\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [63]:
#POS tagging
def pos(text):
    return TextBlob(text).tags
    #return[(word,tag) for word, tag in TextBlob(text).tags if tag.startswith('NN') or tag.startswith('VB')]

In [65]:
#Apply for POS
pos_output=input_data.text.apply(pos)

In [66]:
pos_output = pd.DataFrame(pos_output)

In [67]:
#Add tags with words
input_data['text'] = pos_output['text'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

In [68]:
#Split train and test
review_train, review_test, label_train, label_test = train_test_split(input_data['text'],input_data['deceptive'], test_size=0.2,random_state=13)

In [69]:
#TF-IDF
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)
#use_idf-->IDF weigts
#Smooth_IDF-->Division of zero is avoided
#Sublinear---> Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).

In [70]:
#fit and transform train
X_train_tf=tf_vect.fit_transform(review_train)

In [71]:
#get headers
X_train_tf=pd.DataFrame(X_train_tf.todense(),columns=tf_vect.get_feature_names())

In [72]:
#ONLY transform test not fit
X_test_tf = tf_vect.transform(review_test)

In [73]:
X_test_tf =pd.DataFrame(X_test_tf.todense(),columns=tf_vect.get_feature_names())

In [74]:
#Grid Serach CV for SVM
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [76]:
svc_param_selection(X_train_tf,label_train,2)

In [79]:
clf = svm.SVC(C=1,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

In [ ]:
#Save Model
with open('vectorizer.pickle', 'wb') as fin:
    pickle.dump(tf_vect, fin)
with open('mlmodel.pickle','wb') as f:
    pickle.dump(clf,f)

In [ ]:
pkl = open('mlmodel.pickle', 'rb')
clf = pickle.load(pkl)   
vec = open('vectorizer.pickle', 'rb')
tf_vect = pickle.load(vec)

In [80]:
metrics.accuracy_score(label_test, pred)

0.834375

In [81]:
confusion_matrix(label_test, pred)

array([[133,  32],
       [ 21, 134]], dtype=int64)

In [82]:
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.86      0.81      0.83       165
    truthful       0.81      0.86      0.83       155

   micro avg       0.83      0.83      0.83       320
   macro avg       0.84      0.84      0.83       320
weighted avg       0.84      0.83      0.83       320



In [126]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

In [127]:
# Training Naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
model1.fit(X_train_tf, label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [128]:
pred_Naive=model1.predict(X_test_tf)

In [129]:
metrics.accuracy_score(label_test, pred_Naive)

0.85

In [130]:
print(classification_report(pred_Naive, label_test))

              precision    recall  f1-score   support

   deceptive       0.91      0.82      0.86       183
    truthful       0.79      0.89      0.84       137

   micro avg       0.85      0.85      0.85       320
   macro avg       0.85      0.86      0.85       320
weighted avg       0.86      0.85      0.85       320

